In [180]:
# This will tell you your current IP address from google colab
!curl api.ipify.org

66.219.236.250

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    14  100    14    0     0    130      0 --:--:-- --:--:-- --:--:--   132


In [3]:
# Install needed python packages
!pip install pymongo

from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

In [2]:
# Download us-counties.csv
!curl -L "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv" > us-counties.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:04 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:05 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:06 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:07 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:08 --:--:--     0
  0 99.9M    0  1371    0     0    151      0   8d 00h  0:00:09   8d 00h   285
  5 99.9M    5 5840k    0     0   603k      0  0:02

In [4]:
# Connect to MongoDB Atlas (insert your own credentials)
# Replace the placeholder with your Atlas connection string
uri = "" # <-- put your own connection string here!

# Set the Stable API version when creating a new client
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [5]:
# Create or get your DB
db_name = "Cluster0"
db = client.get_database(db_name)

Spark SQL Rewrite in MongoDB 1-6

*Redo the SparkSQL assignment in MongoDB using the aggregation pipeline.*

In [10]:
# 1. Write code to define the schema and then read in the dataset
#    (took me 17 minutes!!!)

import json
import csv
import pandas
# Load the CSV file
df = pandas.read_csv('C:\\Users\\BYU Rental\\Desktop\\us-counties.csv')
data = df.to_dict('records')
db.casesdeaths.drop()
db.casesdeaths.insert_many(data)
print("done")


done


In [181]:
# 2. Write code to find the county with the most deaths
pipeline = [
    {"$sort": {"deaths": -1}},
    {"$limit": 1},
    {"$project": {"_id": 0, "state": 1, "county": 1, "deaths": 1}}
]

result = list(db.casesdeaths.aggregate(pipeline))

print(result)



[{'county': 'New York City', 'state': 'New York', 'deaths': 40267.0}]


In [182]:
# 3. Write code to find the county with the most cases
pipeline = [
    {"$sort": {"cases": -1}},
    {"$limit": 1},
    {"$project": {"_id": 0, "state": 1, "county": 1, "cases": 1}}
]

result = list(db.casesdeaths.aggregate(pipeline))
print(result)


[{'county': 'Los Angeles', 'state': 'California', 'cases': 2908425}]


In [183]:
pipeline = [
    {"$match": {"county": "Utah"}},
    {"$sort": {"cases": -1}},
    {"$limit": 1},
    {"$project": {"_id": 0, "state": 1, "county": 1, "cases": 1}}
]

result = list(db.casesdeaths.aggregate(pipeline))
print(result)


[{'county': 'Utah', 'state': 'Utah', 'cases': 203286}]


In [184]:
# 5. Write code to find the death rate for each state and sort the states by death rate descending
pipeline = [
    {"$group": {
        "_id": {
            "state": "$state",
            "county": "$county"
        },
        "max_deaths_per_county": {"$max": "$deaths"},
        "max_cases_per_county": {"$max": "$cases"}
    }},
    {"$group": {
        "_id": "$_id.state",
        "total_deaths": {"$sum": "$max_deaths_per_county"},
        "total_cases": {"$sum": "$max_cases_per_county"}
    }},
    {"$project": {
        "state": "$_id",
        "_id": 0,
        "total_cases": "$total_cases",
        "total_deaths": "$total_deaths",
        "death_rate": {
            "$cond": [
                {"$eq": ["$total_cases", 0]},
                0,
                {"$multiply": [{"$divide": ["$total_deaths", "$total_cases"]}, 100]}
            ]
        }
    }},
    {"$sort": {"death_rate": -1}}
]

result = list(db.casesdeaths.aggregate(pipeline))
for doc in result:
        print(f"State: {doc['state']}, Death Rate per State: {doc['death_rate']:.2f}")




State: Pennsylvania, Death Rate per State: 1.57
State: Mississippi, Death Rate per State: 1.56
State: Alabama, Death Rate per State: 1.50
State: Arizona, Death Rate per State: 1.49
State: Nevada, Death Rate per State: 1.47
State: Georgia, Death Rate per State: 1.47
State: Michigan, Death Rate per State: 1.46
State: New Jersey, Death Rate per State: 1.45
State: New Mexico, Death Rate per State: 1.45
State: Ohio, Death Rate per State: 1.42
State: Missouri, Death Rate per State: 1.41
State: Connecticut, Death Rate per State: 1.40
State: Indiana, Death Rate per State: 1.38
State: Louisiana, Death Rate per State: 1.37
State: West Virginia, Death Rate per State: 1.37
State: Arkansas, Death Rate per State: 1.36
State: Florida, Death Rate per State: 1.36
State: Massachusetts, Death Rate per State: 1.32
State: Maryland, Death Rate per State: 1.31
State: New York, Death Rate per State: 1.31
State: Texas, Death Rate per State: 1.30
State: Oklahoma, Death Rate per State: 1.29
State: Tennessee, Dea

In [185]:
# 6. Write code to something else interesting with this data – your choice
pipeline = [
    {"$match": {"county": "Snohomish"}},
    {"$group": {
        "_id": "$county",
        "total_cases": {"$sum": "$cases"},
        "count": {"$sum": 1}  
    }},
    {"$project": {
        "county": "$_id",
        "_id": 0,
        "average_cases": {"$divide": ["$total_cases", "$count"]}
    }}
]

result = list(db.casesdeaths.aggregate(pipeline))
for doc in result:
    print(f"County: {doc['county']}, Average Cases: {doc['average_cases']:.2f}")


County: Snohomish, Average Cases: 44398.22


In this next part we will get experience using MongoDB's aggegregation pipeline's $lookup stage to join collections in MongoDb. Specifically we'll join to our **cases/deaths data** with **[vaccination data](https://ourworldindata.org/us-states-vaccinations#what-share-of-the-population-has-completed-the-initial-vaccination-protocol)** and **[total population data](https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv)**.  First we need to download and ingest the data.



In [9]:
# Get the CSV for covid vaccination data
!curl -L "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/us_state_vaccinations.csv" > "./us_state_vaccinations.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 4804k    0 13710    0     0  15289      0  0:05:21 --:--:--  0:05:21 15301
100 4804k  100 4804k    0     0  3427k      0  0:00:01  0:00:01 --:--:-- 3429k


In [11]:
# Put the vaccinations data into the the DB  (took me 37 seconds)
import json
import csv
import pandas
with open("C:\\Users\\BYU Rental\\Desktop\\us_state_vaccinations.csv") as f:
  dataRows = csv.DictReader(f)
  db.vaccinations.insert_many(dataRows)

df = pandas.read_csv('C:\\Users\\BYU Rental\\Desktop\\us_state_vaccinations.csv')
data = df.to_dict('records')
db.vaccinations.drop()
db.vaccinations.insert_many(data)
print("Done!")


Done!


Use the \$lookup stage of the aggregation pipeline to join the vaccination and cases/deaths collections (you will need to group the covid case data by state) and use the \$out stage to create a new joined collection.

In [186]:
# Create a "joined collection" of data by state for cases, deaths, vaccinations counts
# The cases and deaths data has over 2 million rows. You'll need to summarize it to
# states before you can successfully join it to the other dataset.

# YOUR CODE HERE
res = db.vaccinations.aggregate([
    {
        "$group": {
            "_id": "$location",
            "vaccinated": {
                "$max": "$people_vaccinated" 
                
            }
        }
    },
    {
        "$out": "vaccinated_state"
    }
    
])

list(res)

[]

In [14]:
# Get the total population (Use POPESTIMATE2019)
!curl -L "https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv" > "./co-est2019-alldata.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  225k    0  225k    0     0   199k      0 --:--:--  0:00:01 --:--:--  199k
100 3559k    0 3559k    0     0  2596k      0 --:--:--  0:00:01 --:--:-- 2596k


In [15]:
# Put population data into the DB (took me 10 seconds)
# with open("./co-est2019-alldata.csv", encoding='latin-1') as f:
#   dataRows = csv.DictReader(f)
#   db.population.insert_many(dataRows)

df = pandas.read_csv('C:\\Users\\BYU Rental\\Desktop\\co-est2019-alldata.csv', encoding='latin-1')
data = df.to_dict('records')
db.populations.drop()
db.populations.insert_many(data)
print("Done!")

Done!


Now let's incorrectly assume that vaccinations remove a person from possibility of infection and death. So based on our incorrect assumption, subtract the number who completed the initial vaccination protocol from the population of the state (POPESTIMATE2019 field) and and calculate a case percent and death percent of the population that was not ever vaccinated. 🤔

In [187]:
# How will join the population data into this?
#Summarize the Vaccinations Data: First, ensure that your vaccinations data is summarized by state if it's not already. You should have the total number of individuals who completed the vaccination protocol per state.
#Not correct
res = db.populations.aggregate([
    {
        "$group": {
            "_id": "$STNAME",
            "population": {
                "$max": "$POPESTIMATE2019" 
                
            }
        }
    },
    {
        "$out": "population_state"
    }
    
])

list(res)


[]

In [188]:
pipeline = [
    {
        "$group": {
            "_id": {
                "state": "$state",
                "county": "$county"
            },
            "max_deaths_per_county": {"$max": "$deaths"},
            "max_cases_per_county": {"$max": "$cases"}
        }
    },
    {
        "$group": {
            "_id": "$_id.state",
            "max_deaths": {"$max": "$max_deaths_per_county"},
            "max_cases": {"$max": "$max_cases_per_county"}
        }
    },
    {
        "$merge": {
            "into": "cases_per_state",  
            "whenMatched": "replace",  
            "whenNotMatched": "insert"  
        }
    }
]

db.casesdeaths.aggregate(pipeline)
print("Collection saved")


Collection saved


In [189]:
pipeline = [
    {
        "$lookup": {
            "from": "population_state",
            "localField": "_id",
            "foreignField": "_id",
            "as": "population_data"
        }
    },
    {
        "$unwind": {
            "path": "$population_data",
            "preserveNullAndEmptyArrays": True
        }
    },
    {
        "$lookup": {
            "from": "vaccinated_state",
            "localField": "_id",
            "foreignField": "_id",
            "as": "vaccine_data"
        }
    },
    {
        "$unwind": {
            "path": "$vaccine_data",
            "preserveNullAndEmptyArrays": True
        }
    },
    {
        "$project": {
            "_id": 0,
            "state": "$_id",
            "max_deaths": 1,
            "max_cases": 1,
            "population": "$population_data.population",
            "vaccinated": "$vaccine_data.vaccinated"
        }
    },
    {
        "$merge": {
            "into": "state_data",
            "whenMatched": "replace",
            "whenNotMatched": "insert"
        }
    }
]

db.cases_per_state.aggregate(pipeline)

print("Full Merge")


Full Merge


R1 - Report the highest 10 states and the lowest 10 states ordered by deaths divided by cases. (without subtracting vaccinated - see #5 from the SparkSQL redo)

In [190]:
# Highest 10 states Deaths/Cases (death rate)
pipeline = [
    {
        "$project": {
            "state": 1,
            "death_rate": {
                "$cond": [
                    {"$eq": ["$max_cases", 0]},  
                    0,
                    {"$divide": ["$max_deaths", "$max_cases"]}
                ]
            }
        }
    },
    {"$sort": {"death_rate": -1}},  
    {"$limit": 10}  
]

top_states_by_death_rate = list(db.state_data.aggregate(pipeline))

print("Top 10 States by Death Rate (Deaths/Cases):")
for state in top_states_by_death_rate:
    print(f"State: {state['state']}, Death Rate: {state['death_rate']:.6f}")


Top 10 States by Death Rate (Deaths/Cases):
State: Puerto Rico, Death Rate: 0.062954
State: Puerto Rico, Death Rate: 0.062954
State: Puerto Rico, Death Rate: 0.062954
State: Michigan, Death Rate: 0.019143
State: Michigan, Death Rate: 0.019143
State: Michigan, Death Rate: 0.019143
State: New York, Death Rate: 0.016621
State: New York, Death Rate: 0.016621
State: New York, Death Rate: 0.016621
State: Pennsylvania, Death Rate: 0.016079


In [191]:
# Lowest 10 states Deaths/Cases (death rate)
pipeline = [
    {
        "$group": {
            "_id": "$state",
            "max_deaths": {"$first": "$max_deaths"},  
            "max_cases": {"$first": "$max_cases"}  
        }
    },
    {
        "$project": {
            "state": "$_id",
            "_id": 0,
            "death_rate": {
                "$cond": [
                    {"$eq": ["$max_cases", 0]},
                    None,
                    {"$divide": ["$max_deaths", "$max_cases"]}
                ]
            }
        }
    },
    {"$match": {"death_rate": {"$ne": None}}},
    {"$sort": {"death_rate": 1}},
    {"$limit": 10}
]

bottom_states_by_death_rate = list(db.state_data.aggregate(pipeline))

print("Lowest 10 States by Death Rate (Deaths/Cases):")
for state in bottom_states_by_death_rate:
    print(f"State: {state['state']}, Death Rate: {state['death_rate']:.6f}")





Lowest 10 States by Death Rate (Deaths/Cases):
State: Northern Mariana Islands, Death Rate: 0.003174
State: Utah, Death Rate: 0.004723
State: Alaska, Death Rate: 0.004990
State: Vermont, Death Rate: 0.005054
State: American Samoa, Death Rate: 0.005059
State: North Carolina, Death Rate: 0.005399
State: North Dakota, Death Rate: 0.005556
State: Hawaii, Death Rate: 0.005956
State: Washington, Death Rate: 0.006761
State: Virgin Islands, Death Rate: 0.006826


R2 - Report the highest 10 states and lowest 10 ordered by these new case and death percentages. (with subtracting vaccinated)

In [192]:
# Highest 10 states (cases / (population - vaccinated))
pipeline = [
    {
        "$group": {
            "_id": "$state",
            "max_cases_sum": {"$sum": "$max_cases"},  
            "population": {"$first": "$population"},  
            "vaccinated": {"$first": "$vaccinated"}  
        }
    },
    {
        "$project": {
            "state": "$_id",
            "_id": 0,
            "cases_per_unvaccinated_population": {
                "$cond": [
                    {"$or": [
                        {"$eq": ["$population", 0]},
                        {"$lte": [{"$subtract": ["$population", "$vaccinated"]}, 0]}
                    ]},
                    None,
                    {
                        "$divide": [
                            "$max_cases_sum",
                            {"$subtract": ["$population", "$vaccinated"]}
                        ]
                    }
                ]
            }
        }
    },
    {"$match": {"cases_per_unvaccinated_population": {"$ne": None}}},
    {"$sort": {"cases_per_unvaccinated_population": -1}},
    {"$limit": 10}
]

top_states_by_cases_unvaccinated_ratio = list(db.state_data.aggregate(pipeline))

print("Highest 10 states (cases / (population - vaccinated)):")
for state in top_states_by_cases_unvaccinated_ratio:
    print(f"State: {state['state']}, Ratio: {state['cases_per_unvaccinated_population']:.6f}")

Highest 10 states (cases / (population - vaccinated)):
State: Vermont, Ratio: 1997.826923
State: Maine, Ratio: 5.567479
State: Hawaii, Ratio: 4.537809
State: New Mexico, Ratio: 4.172439
State: Delaware, Ratio: 4.056845
State: Arizona, Ratio: 2.446424
State: Nevada, Ratio: 2.399893
State: New Hampshire, Ratio: 1.957990
State: Alaska, Ratio: 1.614966
State: New Jersey, Ratio: 1.589931


In [193]:
# Lowest 10 states (cases / (population - vaccinated))
pipeline = [
    {
        "$group": {
            "_id": "$state",
            "total_deaths": {"$sum": "$max_deaths"},
            "total_cases": {"$sum": "$max_cases"},
            "population": {"$first": "$population"},  
            "vaccinated": {"$first": "$vaccinated"}
        }
    },
    {
        "$project": {
            "state": "$_id",
            "_id": 0,
            "deaths_per_unvaccinated_population": {
                "$divide": [
                    "$total_deaths",
                    {"$subtract": ["$population", "$vaccinated"]}
                ]
            }
        }
    },
    {"$match": {"deaths_per_unvaccinated_population": {"$gte": 0}}},
    {"$sort": {"deaths_per_unvaccinated_population": 1}},
    {"$limit": 10}
]

bottom_states_by_deaths_unvaccinated_ratio = list(db.state_data.aggregate(pipeline))

print("# Lowest 10 states (cases / (population - vaccinated)):")
for state in bottom_states_by_deaths_unvaccinated_ratio:
    print(f"State: {state['state']}, Ratio: {state['deaths_per_unvaccinated_population']:.6f}")


# Lowest 10 states (cases / (population - vaccinated)):
State: Mississippi, Ratio: 0.002087
State: Georgia, Ratio: 0.002111
State: Louisiana, Ratio: 0.002347
State: Ohio, Ratio: 0.002859
State: Iowa, Ratio: 0.003605
State: West Virginia, Ratio: 0.003746
State: Indiana, Ratio: 0.003778
State: Wyoming, Ratio: 0.003866
State: Arkansas, Ratio: 0.003942
State: South Carolina, Ratio: 0.004063


In [194]:
# Highest 10 states (deaths / (population - vaccinated))
pipeline = [
    {
        "$group": {
            "_id": "$state",
            "max_deaths": {"$first": "$max_deaths"},  
            "population": {"$first": "$population"}, 
            "vaccinated": {"$first": "$vaccinated"}  
        }
    },
    {
        "$project": {
            "state": "$_id",
            "_id": 0,
            "deaths_per_unvaccinated_population": {
                "$cond": [
                    {"$or": [
                        {"$eq": ["$population", 0]},
                        {"$lte": [{"$subtract": ["$population", "$vaccinated"]}, 0]}
                    ]},
                    None,  
                    {
                        "$divide": [
                            "$max_deaths",
                            {"$subtract": ["$population", "$vaccinated"]}
                        ]
                    }
                ]
            }
        }
    },
    {"$match": {"deaths_per_unvaccinated_population": {"$ne": None}}},
    {"$sort": {"deaths_per_unvaccinated_population": -1}},
    {"$limit": 10}
]

top_states_by_deaths_unvaccinated_ratio = list(db.state_data.aggregate(pipeline))

print("Highest 10 states (deaths / (population - vaccinated)):")
for state in top_states_by_deaths_unvaccinated_ratio:
    print(f"State: {state['state']}, Ratio: {state['deaths_per_unvaccinated_population']:.6f}")



Highest 10 states (deaths / (population - vaccinated)):
State: Vermont, Ratio: 3.365385
State: New Mexico, Ratio: 0.017243
State: Maine, Ratio: 0.012924
State: Delaware, Ratio: 0.012720
State: Nevada, Ratio: 0.012442
State: Arizona, Ratio: 0.011007
State: Hawaii, Ratio: 0.009009
State: New Jersey, Ratio: 0.008002
State: New Hampshire, Ratio: 0.005877
State: Illinois, Ratio: 0.005668


In [195]:
# Lowest 10 states (deaths / (population - vaccinated))
pipeline = [
    {
        "$group": {
            "_id": "$state",
            "max_deaths": {"$first": "$max_deaths"},  
            "population": {"$first": "$population"},  
            "vaccinated": {"$first": "$vaccinated"}  
        }
    },
    {
        "$project": {
            "state": "$_id",
            "_id": 0,
            "deaths_per_unvaccinated_population": {
                "$cond": [
                    {"$or": [
                        {"$eq": ["$population", 0]},  
                        {"$lte": [{"$subtract": ["$population", "$vaccinated"]}, 0]}  
                    ]},
                    None,
                    {
                        "$divide": [
                            "$max_deaths",
                            {"$subtract": ["$population", "$vaccinated"]}
                        ]
                    }
                ]
            }
        }
    },
    {"$match": {"deaths_per_unvaccinated_population": {"$ne": None}}},
    {"$sort": {"deaths_per_unvaccinated_population": 1}},
    {"$limit": 10}
]

bottom_states_by_deaths_unvaccinated_ratio = list(db.state_data.aggregate(pipeline))

print("Lowest 10 states (deaths / (population - vaccinated)):")
for state in bottom_states_by_deaths_unvaccinated_ratio:
    print(f"State: {state['state']}, Ratio: {state['deaths_per_unvaccinated_population']:.6f}")


Lowest 10 states (deaths / (population - vaccinated)):
State: Mississippi, Ratio: 0.000696
State: Georgia, Ratio: 0.000704
State: Louisiana, Ratio: 0.000782
State: Ohio, Ratio: 0.000953
State: Iowa, Ratio: 0.001202
State: West Virginia, Ratio: 0.001249
State: Indiana, Ratio: 0.001259
State: Wyoming, Ratio: 0.001289
State: Arkansas, Ratio: 0.001314
State: South Carolina, Ratio: 0.001354


In [196]:
client.close()